In [1]:
from pymongo import MongoClient
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing

client = MongoClient("localhost", 27017, maxPoolSize=50)
db = client.sc
collection = db['messages']
cursor = collection.find({})
data = []
labels = []

for document in cursor:        
        data.append(document['text'])
        labels.append(document['user'])

tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
one_hot_results = tokenizer.texts_to_matrix(data, mode='binary')

lb = preprocessing.LabelBinarizer()
labels = lb.fit_transform(labels)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))



Using TensorFlow backend.


Found 553 unique tokens.


In [2]:
labels

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [2]:
from keras.preprocessing.sequence import pad_sequences
import numpy as np 

maxlen = 100
data = pad_sequences(one_hot_results, maxlen=maxlen)
labels = np.asarray(labels)

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

training_samples = 200
validation_samples = 1000
max_words = 10000

x_train = data[:training_samples]
y_train = labels[:training_samples]

x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Shape of data tensor: (619836, 100)
Shape of label tensor: ()


IndexError: too many indices for array

In [12]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

embedding_dim = 64

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 64)           640000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                204832    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 844,865
Trainable params: 844,865
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train,epochs=10,batch_size=32,
validation_data=(x_val, y_val))
model.save_weights('pre_trained.h5')

Train on 200 samples, validate on 1000 samples
Epoch 1/10


ValueError: could not convert string to float: 'U02AKHZGV'